In [1]:
import struct
import pandas as pd
import xml.etree.ElementTree as ET
import json
import os
import numpy as np
from datetime import datetime
import pyproj

In [13]:
# Soecifu the taskdata folder

data_folder = r'C:/dev/agri_analysis/data/taskdata' 
out_folder = r'C:/dev/agri_analysis/data/taskdata_out2'
# Create output folder if it doesn't exist
os.makedirs(out_folder, exist_ok=True)

# Buffer to add around the farm boundaries (in degrees)
# 0.02 deg is approx 2km. This accounts for headlands/approach roads.
GEO_BUFFER = 0.02 

# DEFINE YOUR LOCAL PROJECTION HERE for spped and mapping purposes
# EPSG:25832 = ETRS89 / UTM zone 32N (Standard for Denmark)
# EPSG:32632 = WGS 84 / UTM zone 32N (International standard)
target_crs = "EPSG:25832"

## XML task file reading

In [14]:
def parse_isobus_taskdata(data_folder):
    # 1. Find TASKDATA.XML
    taskdata_path = None
    for file in os.listdir(data_folder):
        if file.upper() == 'TASKDATA.XML':
            taskdata_path = os.path.join(data_folder, file)
            break
            
    if taskdata_path is None:
        print("Error: TASKDATA.XML not found.")
        return None, None

    try:
        tree = ET.parse(taskdata_path)
        root = tree.getroot()
    except Exception as e:
        print(f"Error parsing XML: {e}")
        return None, None

    # --- 1. Map Products (PDT) ---
    products = {}
    for pdt in root.findall(".//PDT"):
        products[pdt.attrib.get('A')] = pdt.attrib.get('B')

    # --- 2. Store Field Geometries (PFD) ---
    field_shapes = {}   # PFD_ID -> Coordinates
    field_names_map = {} # PFD_ID -> Name

    for pfd in root.findall(".//PFD"):
        pfd_id = pfd.attrib.get('A')
        field_names_map[pfd_id] = pfd.attrib.get('C')
        
        # Extract Geometry (Polygon)
        # We take the first valid polygon found
        for pln in pfd.findall("PLN"):
            for lsg in pln.findall("LSG"):
                coordinates = []
                for pnt in lsg.findall("PNT"):
                    try:
                        # XML Attributes are strings, convert to float
                        lat = float(pnt.attrib.get('C'))
                        lon = float(pnt.attrib.get('D'))
                        coordinates.append([lon, lat])
                    except: continue
                
                if len(coordinates) > 2:
                    # Close the polygon loop
                    if coordinates[0] != coordinates[-1]:
                        coordinates.append(coordinates[0])
                    field_shapes[pfd_id] = coordinates
                    break # Stop after first valid shape

    # --- 3. Build Tasks & Generate GeoJSON ---
    tasks_list = []
    geojson_features = []

    for tsk in root.findall(".//TSK"):
        task_id = tsk.attrib.get('A')
        field_ref = tsk.attrib.get('E')
        
        # Get Attributes
        field_name = field_names_map.get(field_ref, f"Unknown ({field_ref})")
        
        # Get Log File
        tlg = tsk.find("TLG")
        log_filename = tlg.attrib.get('A') if tlg is not None else None

        # Get Crop
        crop_name = "Unknown"
        pan = tsk.find("PAN")
        if pan is not None:
            pdt_ref = pan.attrib.get('A')
            crop_name = products.get(pdt_ref, pdt_ref)
            
        # --- NEW: GET YEAR ---
        # Find all TIM tags and get the earliest start time
        start_times = []
        for tim in tsk.findall("TIM"):
            start_str = tim.attrib.get('A') # Format: 2023-08-16T11:34:57...
            if start_str:
                try:
                    # Parse ISO format (handle timezone if present, simplistically here)
                    dt = datetime.fromisoformat(start_str.replace('Z', '+00:00'))
                    start_times.append(dt)
                except: pass
        
        task_year = start_times[0].year if start_times else None

        # -- Build DataFrame Entry --
        if log_filename: # Only save interesting tasks with logs
            tasks_list.append({
                'TaskID': task_id,
                'Year': task_year,
                'Crop': crop_name,
                'FieldName': field_name,
                'FieldID': field_ref,
                'LogFilename': log_filename
            })

            # -- Build GeoJSON Feature --
            if field_ref in field_shapes:
                feature = {
                    "type": "Feature",
                    "properties": {
                        "TaskID": task_id,
                        "Year": int(task_year) if task_year else None,
                        "FieldName": field_name,
                        "Crop": crop_name,
                        "LogFilename": log_filename 
                    },
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [field_shapes[field_ref]]
                    }
                }
                geojson_features.append(feature)

    # --- Output ---
    geojson_output = {
        "type": "FeatureCollection", 
        "features": geojson_features
    }
    df_tasks = pd.DataFrame(tasks_list)
    
    return geojson_output, df_tasks




In [15]:
# --- EXECUTION ---
geojson, df_tasks = parse_isobus_taskdata(data_folder)

if df_tasks is not None:
    # Save CSV
    csv_path = os.path.join(out_folder, 'task_index.csv')
    df_tasks.to_csv(csv_path, index=False)
    
    # Save GeoJSON
    geo_path = os.path.join(out_folder, 'harvest_tasks.geojson')
    with open(geo_path, 'w') as f:
        json.dump(geojson, f)
        
    print(f"Success. Index saved to {csv_path}")
    print(f"Sample:\n{df_tasks[['Year', 'Crop', 'FieldName']].head()}")

Success. Index saved to C:/dev/agri_analysis/data/taskdata_out2\task_index.csv
Sample:
   Year              Crop      FieldName
0  2024  RA - Raps / rybs  021-0, Monica
1  2024  RA - Raps / rybs  021-0, Monica
2  2025        HV - Hvede         Import
3  2025        HV - Hvede         Import
4  2025       VB - Vårbyg         Import


## Binary file reading

In [9]:
# --- HELPER: PACKET CLASSIFIER ---
def classify_packet(gap_size):
    """
    Determines the machine state based on the size of the proprietary header
    preceding the data record.
    """
    if gap_size == 25:
        return "Harvest (Mode A)"
    elif gap_size == 10:
        return "Transport (Mode B)"
    elif gap_size == 0:
        return "Contiguous" # Rare, usually start of file
    else:
        return f"Transition ({gap_size}b)"

# --- HELPER: DYNAMIC BOUNDING BOX ---
def get_farm_bounding_box(taskdata_path):
    print(f"Reading Farm Geometry from: {taskdata_path}")
    try:
        tree = ET.parse(taskdata_path)
        root = tree.getroot()
        lats, lons = [], []
        for pnt in root.findall(".//PFD//PNT"):
            try:
                lats.append(float(pnt.attrib.get('C')))
                lons.append(float(pnt.attrib.get('D')))
            except: continue
            
        if not lats: return 54.5, 58.0, 8.0, 15.0
        return min(lats)-GEO_BUFFER, max(lats)+GEO_BUFFER, min(lons)-GEO_BUFFER, max(lons)+GEO_BUFFER
    except: return 54.5, 58.0, 8.0, 15.0

# --- HELPER: METADATA & DEFS ---
def load_metadata(taskdata_path):
    try:
        tree = ET.parse(taskdata_path)
        root = tree.getroot()
        pdt_map = {pdt.attrib.get('A'): pdt.attrib.get('B') for pdt in root.findall(".//PDT")}
        meta_map = {}
        for tsk in root.findall(".//TSK"):
            tlg = tsk.find(".//TLG")
            pan = tsk.find(".//PAN")
            tim = tsk.find(".//TIM")
            if tlg is not None:
                log_id = tlg.attrib.get('A')
                crop = pdt_map.get(pan.attrib.get('A'), 'Unknown') if pan is not None else 'Unknown'
                start = tim.attrib.get('A') if tim is not None else None
                meta_map[log_id] = {'Crop': crop, 'Start_Time': start}
                meta_map[log_id + '.bin'] = {'Crop': crop, 'Start_Time': start}
        return meta_map
    except: return {}

def get_dlv_defs(xml_path):
    try:
        tree = ET.parse(xml_path)
        return [elem.attrib.get('A') for elem in tree.findall(".//DLV")]
    except: return []

# --- CORE: ENRICHED FORENSIC SCANNER (Corrected: Unsigned 'I') ---
def forensic_scan(bin_path, definitions, metadata, bounds):
    min_lat, max_lat, min_lon, max_lon = bounds
    filename = os.path.basename(bin_path)
    meta = metadata.get(filename, {'Crop': 'Unknown', 'Start_Time': None})
    
    with open(bin_path, 'rb') as f:
        content = f.read()

    total_len = len(content)
    cursor = 0
    valid_rows = []
    last_valid_end = 0
    
    # Map DDI IDs to human names
    ddi_lookup = {'0054': 'Yield_Mass', '0095': 'Yield_Vol', '018D': 'Speed', '0063': 'Moisture'}

    while cursor < total_len - 50:
        match_found = False
        bytes_consumed = 0
        
        try:
            # Header: Time (Unsigned Long 'L'), Lat (Signed 'i'), Lon (Signed 'i')
            # Lat/Lon must remain Signed because coordinates can be negative
            time_ms, lat_raw, lon_raw = struct.unpack('<Lii', content[cursor:cursor+12])
            lat = lat_raw * 1e-7
            lon = lon_raw * 1e-7
            
            if (min_lat < lat < max_lat) and (min_lon < lon < max_lon):
                count = len(definitions)
                payload_len = count * 4
                
                if cursor + 12 + payload_len <= total_len:
                    # CRITICAL FIX: Use 'I' (Unsigned Int) for sensor payload
                    # This prevents 0x80000000 from becoming a negative number
                    values = struct.unpack(f"{count}I", content[cursor+12 : cursor+12+payload_len])
                    
                    row_data = {'Time_ms': time_ms, 'Latitude': lat, 'Longitude': lon}
                    for i, val in enumerate(values):
                        ddi = definitions[i]
                        row_data[ddi_lookup.get(ddi, f'DDI_{ddi}')] = val
                    
                    bytes_consumed = 12 + payload_len
                    match_found = True
        except: pass

        if match_found:
            gap_size = cursor - last_valid_end
            
            row_data['File'] = filename
            row_data['Crop'] = meta['Crop']
            row_data['Start_Time_Str'] = meta['Start_Time']
            row_data['Gap_Bytes'] = gap_size
            row_data['Packet_Type'] = classify_packet(gap_size)

            valid_rows.append(row_data)

            cursor += bytes_consumed
            last_valid_end = cursor
        else:
            cursor += 1

    return pd.DataFrame(valid_rows)

In [11]:
# --- EXECUTION ---
taskdata_xml = os.path.join(data_folder, 'TASKDATA.XML')
farm_bounds = get_farm_bounding_box(taskdata_xml)
metadata_map = load_metadata(taskdata_xml)

all_data = []

print("Starting Enriched Forensic Scan...")

for filename in os.listdir(data_folder):
    if filename.upper().startswith("TLG") and filename.upper().endswith(".BIN"):
        xml_full = os.path.join(data_folder, filename.rsplit('.', 1)[0] + ".xml")
        bin_full = os.path.join(data_folder, filename)
        
        if os.path.exists(xml_full):
            
            defs = get_dlv_defs(xml_full)
            
            df = forensic_scan(bin_full, defs, metadata_map, farm_bounds)
 
            if not df.empty:
                all_data.append(df)


# --- POST-PROCESSING (Tuned for Sparse Data) ---
if all_data:
    print("Processing Metrics...")
    
    # 1. Concatenate strictly in order
    master = pd.concat(all_data, ignore_index=True)
    
    # Remove duplicates to prevent time errors
    master.drop_duplicates(subset=['File', 'Time_ms', 'Latitude', 'Longitude'], keep='first', inplace=True)

    # 2. Time Reconstruction
    def reconstruct_continuous_time(group):
        diffs = group['Time_ms'].diff().fillna(0)
        wrap_mask = diffs < -1000 
        raw_max = group['Time_ms'].max()
        if 59000 < raw_max < 61000: cycle_size = 60000 
        elif 65000 < raw_max < 66000: cycle_size = 65536
        else: cycle_size = raw_max 
        group['Wrap_Offset'] = wrap_mask.cumsum() * cycle_size
        group['Time_Continuous'] = group['Time_ms'] + group['Wrap_Offset']
        return group

    master = master.groupby('File', group_keys=False).apply(reconstruct_continuous_time)
    master['Task_Start'] = pd.to_datetime(master['Start_Time_Str'])
    master['Datetime'] = master['Task_Start'] + pd.to_timedelta(master['Time_Continuous'], unit='ms')

    # 3. Sensor Cleaning & Scaling (Corrected for your Units)
    def clean_and_scale(series, scale_factor=1.0):
        s = pd.to_numeric(series, errors='coerce').fillna(0)
        # Fix Unsigned Wrap & Clamp Negative Noise
        s = np.where(s > 2147483647, 0, s)
        s = np.where(s < 0, 0, s)
        return s / scale_factor

    # SPEED: Raw is ~1000. Unit is mm/s. Divide by 1000 to get m/s.
    if 'Speed' in master.columns:
        master['Speed'] = clean_and_scale(master['Speed'], scale_factor=1000.0)

    # MOISTURE: Raw is ~990. Unit is 0.01%. Divide by 100 to get %.
    if 'Moisture' in master.columns:
        master['Moisture'] = clean_and_scale(master['Moisture'], scale_factor=100.0)

    master['Raw_Mass'] = clean_and_scale(master['Yield_Mass'], scale_factor=1.0)
    master['Raw_Vol']  = clean_and_scale(master['Yield_Vol'], scale_factor=1.0)
    master['Status_Code'] = master['Raw_Mass'].astype(int) & 0xFF
    master['Density_kg_L'] = np.where(master['Raw_Vol'] > 1000, (master['Raw_Mass'] / master['Raw_Vol']), 0)

    # 4. GPS SPEED (Window Fixed for 20m Steps)
    print("  -> Reconstructing GPS Speed...")
    import pyproj
    transformer = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:25832", always_xy=True)
    xx, yy = transformer.transform(master['Longitude'].values, master['Latitude'].values)
    master['UTM_Easting'] = xx
    master['UTM_Northing'] = yy

    # UPDATED WINDOW: 5 samples (~100m) is enough since each step is huge (20m)
    SPEED_WINDOW = 5 
    
    grp = master.groupby('File')
    
    dx = grp['UTM_Easting'].diff(SPEED_WINDOW)
    dy = grp['UTM_Northing'].diff(SPEED_WINDOW)
    dist_m = np.sqrt(dx**2 + dy**2)
    dt_s = grp['Time_Continuous'].diff(SPEED_WINDOW) / 1000.0
    dt_s = dt_s.where(dt_s > 0.1, np.nan) 
    
    master['GPS_Speed'] = dist_m / dt_s
    master['GPS_Speed'] = master['GPS_Speed'].bfill().fillna(0).clip(0, 25)

    # 5. Yield Calc (Using SENSOR SPEED if available, else GPS)
    # Since your GPS points are sparse (20m gaps), Sensor Speed (mm/s) is likely more accurate for Yield.
    HEADER_WIDTH = 9.0 
    master['Mass_kg_s'] = master['Raw_Mass'] / 1_000_000
    
    # Use Sensor Speed if it's valid (>0.1), otherwise fallback to GPS
    # This prevents the "20m jump" from messing up the yield
    use_speed = np.where(master['Speed'] > 0.1, master['Speed'], master['GPS_Speed'])
    
    master['Yield_T_Ha'] = (master['Mass_kg_s'] * 10.0) / (use_speed * HEADER_WIDTH)
    master['Yield_T_Ha'] = master['Yield_T_Ha'].fillna(0).replace([np.inf, -np.inf], 0)

    # --- SAVING ---
    out_csv = os.path.join(out_folder, 'UNIVERSAL_ENRICHED_DATASET.csv')
    
    cols = [
        'Datetime', 'Latitude', 'Longitude', 
        'UTM_Easting', 'UTM_Northing',
        'File', 'Crop', 'Packet_Type', 'Gap_Bytes',   
        'Yield_T_Ha', 'Density_kg_L', 
        'GPS_Speed', 'Speed', 'Moisture',
        'Status_Code', 'Raw_Mass', 'Raw_Vol'
    ]
    
    valid_cols = [c for c in cols if c in master.columns]
    master[valid_cols].to_csv(out_csv, index=False)
    
    print(f"\nSUCCESS.")
    print(f"Dataset saved to: {out_csv}")
    print(f"Speed Window: {SPEED_WINDOW} samples")
    if 'Speed' in master.columns:
        print(f"Sensor Speed Scaled (Factor 1000): Max {master['Speed'].max():.2f} m/s")
    if 'Moisture' in master.columns:
        print(f"Moisture Scaled (Factor 100): Max {master['Moisture'].max():.2f} %")

else:
    print("No valid data found.")

Reading Farm Geometry from: ./data/taskdata\TASKDATA.XML
Starting Enriched Forensic Scan...
Processing Metrics...


C:\Users\holmes\AppData\Local\Temp\ipykernel_20672\379185482.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  master = master.groupby('File', group_keys=False).apply(reconstruct_continuous_time)


  -> Reconstructing GPS Speed...

SUCCESS.
Dataset saved to: ./data/taskdata_out\UNIVERSAL_ENRICHED_DATASET.csv
Speed Window: 5 samples
Sensor Speed Scaled (Factor 1000): Max 2145386.50 m/s
Moisture Scaled (Factor 100): Max 21472299.87 %
